In [1]:
!pip install pytrends


In [2]:
from pytrends.request import TrendReq
from pylab import rcParams
from ipywidgets import interact, interactive, fixed, interact_manual

import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt

import ipywidgets as widgets
import pandas as pd
import statsmodels.api as sm
import matplotlib
import pandas as pd
import plotly.offline as py
import plotly.graph_objs as go
import seaborn as sns

plt.rcParams.update({'font.size': 9})

sns.set(style="darkgrid")

pytrend = TrendReq()

In [4]:
def get_google_trends_data(list_product, time_start, time_end, state=None, country='PE'):
    if state:
        sigla='{}-{}'.format(country, state)
    else:
        sigla='{}'.format(country)

    data_composer = '{} {}'.format(time_start, time_end)
    pytrend.build_payload(kw_list=list_product,
                                  geo=sigla,
                                  cat=0,
                                  timeframe=data_composer)

    return pytrend.interest_over_time()

In [5]:
def get_trends_data(samples, industry_label, start_date, end_date):
    list_product = samples['sample']
    dataset = get_google_trends_data(list_product=list_product, time_start=start_date, time_end=end_date)
    dataset = dataset.reset_index()
    dataset['Tipo'] = industry_label
    return dataset

# Definir los samples y extraer los datos
test_samples1 = {'sample': ["Keiko Fujimori", "Antauro Humala", "César Acuña", "Verónika Mendoza", "Hernando de Soto"]}
test_samples2 = {'sample': ["Corrupción", "Economía", "Etnocacerismo", "Reforma agraria", "Libertad de expresión"]}
test_samples3 = {'sample': ["El Comercio", "RPP", "La República", "América TV", "Perú 21"]}
test_samples4 = {'sample': ["yape", "plin", "bim", "tunki"]}


dataset1 = get_trends_data(test_samples1, 'Canditatos', '2020-01-01', '2024-09-30')
dataset2 = get_trends_data(test_samples2, 'Palabras relacionadas a Canditados', '2020-01-01', '2024-09-30')
dataset3 = get_trends_data(test_samples3, 'Publicidad', '2020-01-01', '2024-09-30')
dataset4 = get_trends_data(test_samples4, 'Billetera Digital', '2020-01-01', '2024-09-30')

# Concatenar ambos DataFrames en uno solo
final_dataset = pd.concat([dataset1, dataset2, dataset3, dataset4], ignore_index=True)

# Convertir a formato largo usando melt y mantener 'Tipo'
df_google_trends = final_dataset.melt(id_vars=['date', 'Tipo'], var_name='term', value_name='score')

# Eliminar la columna 'isPartial' y filas con valores nulos
df_google_trends = df_google_trends[df_google_trends['term'] != 'isPartial'].dropna(subset=['score'])

# Mostrar la nueva estructura
print(df_google_trends)

# Exportar a CSV si es necesario
df_google_trends.to_csv('df_google_trends_final.csv', sep=',', index=False)

            date               Tipo            term score
0     2019-12-29         Canditatos  Keiko Fujimori  17.0
1     2020-01-05         Canditatos  Keiko Fujimori  10.0
2     2020-01-12         Canditatos  Keiko Fujimori   8.0
3     2020-01-19         Canditatos  Keiko Fujimori   8.0
4     2020-01-26         Canditatos  Keiko Fujimori  42.0
...          ...                ...             ...   ...
19915 2024-09-01  Billetera Digital           tunki   1.0
19916 2024-09-08  Billetera Digital           tunki   1.0
19917 2024-09-15  Billetera Digital           tunki   1.0
19918 2024-09-22  Billetera Digital           tunki   1.0
19919 2024-09-29  Billetera Digital           tunki   1.0

[4731 rows x 4 columns]
